# Capítulo 8: Reducción de Dimensionalidades

## <span style="color:green">1. PCA</span>

PCA=Principal Component Analysis

In [3]:
#Importa los librerías pandas, numpy, matplotlib, os. Ya te la sabes, babyyy
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import os 

In [4]:
#Trae los datos que necesitamos
candidates = {'gmat': [780,750,690,710,680,730,690,720,740,690,610,690,710,680,770,610,580,650,540,590,620,600,550,550,570,670,660,580,650,660,640,620,660,660,680,650,670,580,590,690],
              'gpa': [4,3.9,3.3,3.7,3.9,3.7,2.3,3.3,3.3,1.7,2.7,3.7,3.7,3.3,3.3,3,2.7,3.7,2.7,2.3,3.3,2,2.3,2.7,3,3.3,3.7,2.3,3.7,3.3,3,2.7,4,3.3,3.3,2.3,2.7,3.3,1.7,3.7],
              'work_experience': [3,4,3,5,4,6,1,4,5,1,3,5,6,4,3,1,4,6,2,3,2,1,4,1,2,6,4,2,6,5,1,2,4,6,5,1,2,1,4,5],
              'admitted': [1,1,0,1,0,1,0,1,1,0,0,1,1,0,1,0,0,1,0,0,1,0,0,0,0,1,1,0,1,1,0,0,1,1,1,0,0,0,0,1]
              }
df = pd.DataFrame(candidates,columns= ['gmat', 'gpa','work_experience','admitted'])
df.head()

,gmat,gpa,work_experience,admitted
0,780,4.0,3,1
1,750,3.9,4,1
2,690,3.3,3,0
3,710,3.7,5,1
4,680,3.9,4,0


In [5]:
#Separa el dataframe en los datos que vamos a utilizar para predecir y los datos predichos
x = df[['gpa','gmat','work_experience']]
y = df['admitted']

In [7]:
#Una vez teniendo nuestros datos centramos x restándole su media.
x_centrada  = x - x.mean(axis=0)
#Recordemos que SVD descompone X en 3 matrices U, E y V por lo que utilizamos la función de numpy svd.
U, E, V = np.linalg.svd(x_centrada)
#V es la que contiene los vectores con los componentes principales
#para obtener los primeros 2 simplemente transponemos sus primeras 2 columnas.
pc1 = V.T[:,0]
pc2 = V.T[:,1]
pc1,pc2

(array([-0.00576901, -0.99991795, -0.01143706]),
 array([ 0.16747873, -0.01224184,  0.98579968]))

Para proyectar nuestros datos en el hiperplano y obtener nuestra matriz X de dimensiones reducidas
tenemos que XNUEVA = XW donde X es nuestra matriz de datos original y W es la matriz 
que contiene nuestros vectores de componentes principales.

In [8]:
W = V.T[:, :2]
x_nueva = x_centrada.dot(W)
x_nueva.head()

,0,1
0,-125.990022,-1.809869
1,-96.003344,-0.473562
2,-35.993368,-0.825338
3,-56.016909,0.968416
4,-26.009087,0.383367


### <span style="color:blue">1.1 Con Scikit</span>

In [10]:
#Importa PCA
from sklearn.decomposition import PCA
#Genera el objeto 
pca = PCA(n_components=2)
x_nueva = pca.fit_transform(x)
x_nueva[0:5]

array([[125.99002225,   1.80986853],
       [ 96.00334382,   0.4735615 ],
       [ 35.9933682 ,   0.82533799],
       [ 56.01690897,  -0.96841606],
       [ 26.00908713,  -0.38336734]])

In [11]:
#Calula la distribución de la varianza  
pca.explained_variance_ratio_

array([9.99258771e-01, 6.86720501e-04])

Esta distribución nos indica la proporción de varianza que cada componente principal contiene en comparación al set de datos original.

In [12]:
# Crea  un modelo donde tengas al menos el 90% de la varianza consevada
pca = PCA(n_components=0.90)
x_nueva = pca.fit_transform(x)
x_nueva[0:5]

array([[125.99002225],
       [ 96.00334382],
       [ 35.9933682 ],
       [ 56.01690897],
       [ 26.00908713]])

***

In [13]:
#descomprime los datos con la función inverse_transform
pca = PCA(n_components=1)
x_nueva = pca.fit_transform(x)
x_recuperada = pca.inverse_transform(x_nueva)
#Resultado de la descompresión
x_recuperada[0:5]

array([[  3.82183783, 779.97968511,   4.86595537],
       [  3.64884435, 749.99546701,   4.52299595],
       [  3.30264614, 689.99041504,   3.83665829],
       [  3.41816217, 710.01231294,   4.06566872],
       [  3.24504671, 680.00695316,   3.72246748]])

Se pierde fidelidad porque no explica la variación en todos los datos pero es suficiente para entrenar al algoritmo

***

### <span style="color:blue">1.2 PCA Incremental</span>

In [15]:
#Importar IncrementalPCA
from sklearn.decomposition import IncrementalPCA
#Divide los datos en 3, es decir, 3 subsets. 
subsets = 3
ipca = IncrementalPCA(n_components=1)
for subset in np.array_split(x, subsets):
    ipca.partial_fit(subset)
    
x_nueva = ipca.transform(x)
x_nueva[0:5]

array([[125.99002318],
       [ 96.00334408],
       [ 35.99336862],
       [ 56.01690849],
       [ 26.00908696]])

***

## <span style="color:green">2. Kernel PCA</span>

In [17]:
#Importar KernelPCA
from sklearn.decomposition import KernelPCA
#Hacer un rbf= Radial Basis function 
pca_rbf = KernelPCA(n_components=2, kernel="rbf")

In [19]:
x_nueva=pca_rbf.fit_transform(x)
#Visualiza los resultado 
x_nueva[0:5]

array([[-0.03746614, -0.04697463],
       [-0.03746614, -0.04697463],
       [-0.09203604, -0.18169782],
       [-0.06508374, -0.0952682 ],
       [-0.33597196,  0.80053796]])

### <span style="color:blue">2.1 Seleccionar un Kernel</span>

In [21]:
#Importar GridSearchCV
from sklearn.model_selection import GridSearchCV
#Importar LogisticRegressin
from sklearn.linear_model import LogisticRegression
#Importar Pipeline
from sklearn.pipeline import Pipeline

#Armar un pipeline que pase por KernelPCA y LogisticRegression
clf = Pipeline([
    ("pca", KernelPCA(n_components=2)),
    ("reg", LogisticRegression())
])

In [23]:
#Corre el GridSearch
grid = [{
    "pca__gamma": np.linspace(0.1,0.5,10),
    "pca__kernel": ["rbf","sigmoid"]
}]

In [24]:
#Ejecuta el GridSearch
search = GridSearchCV(clf, grid)
search.fit(x,y)

GridSearchCV(estimator=Pipeline(steps=[('pca', KernelPCA(n_components=2)),
                                       ('reg', LogisticRegression())]),
             param_grid=[{'pca__gamma': array([0.1       , 0.14444444, 0.18888889, 0.23333333, 0.27777778,
       0.32222222, 0.36666667, 0.41111111, 0.45555556, 0.5       ]),
                          'pca__kernel': ['rbf', 'sigmoid']}])

In [25]:
#Imprime el mejor parámetro que equivale al PCA que será la mejor regresión logística
print(search.best_params_)

{'pca__gamma': 0.14444444444444446, 'pca__kernel': 'rbf'}


## <span style="color:green">2. LLE</span>

LLE= Locally Linear Embedding (No es mi favorita 😴)

Es un método de reducción de dimensionalidad no lineal y **no depende de proyecciones**

Funciona midiendo como cada instancia de entrenamiento se relaciona linealmente con sus instancias vecinas y busca una representación lineal de pocas dimensiones del set de datos donde estas relaciones entre instancias cercanas o vecinas están bien preservadas. 

In [28]:
#Importar LocallyLinearEmbedding
from sklearn.manifold import LocallyLinearEmbedding

#Seleccionar el número de dimensiones, componentes y "vecinos" 
lle = LocallyLinearEmbedding(n_components=2, n_neighbors=5)
x_nueva = lle.fit_transform(x)
x_nueva[0:5]

array([[ 0.17455358,  0.54429708],
       [ 0.16458675,  0.31468021],
       [ 0.14479511, -0.15096839],
       [ 0.151172  ,  0.00563436],
       [ 0.11317562, -0.13502106]])